NeuralProphet 有许多超参数需要用户指定。如果未指定，则将使用这些超参数的默认值。查看 forecaster.py 的 API 文档中的 NeuralProphet 类，了解有关所有超参数的详细信息。

# 1. 模型基础参数

Forecast horizon
> n_forecasts 是预测范围的大小。默认值 1 表示模型预测未来一步。

Autoregression
> n_lags 定义是否启用 AR-Net（如果 n_lags > 0）。如果可能，通常建议 n_lags 的值大于 n_forecasts，因为 FFNN 最好遇到至少 n_forecasts 长度的过去，以便预测未来 n_forecasts。因此，n_lags 确定应该考虑自回归依赖关系的过去。这可以是根据领域专业知识或实证分析选择的值。


# 2. 模型训练相关参数

NeuralProphet采用随机梯度下降法进行拟合——更准确地说，是使用AdamW优化器和单周期学习率策略进行训练。

learning_rate
> 学习率 若未指定learning_rate参数，系统会自动执行学习率范围测试以确定最佳学习率

epochs（训练轮次）、loss_func（损失函数）和optimizer（优化器）
> 当这些参数未明确定义时，epochs和loss_func会根据数据集大小自动设置，其设置原则是将总训练步数控制在1000至4000步范围内
> NeuralProphet支持设置两种不同的优化器：AdamW和SDG（随机梯度下降）
> 默认损失函数是“SmoothL1Loss”损失，它被认为是对异常值的鲁棒性。但是，您可以自由选择标准 MSE 或任何其他 PyTorch torch.nn.modules.loss 损失函数

# 3. 增加模型的深度

ar_layers 定义了整个模型中 AR-Net 的隐藏层数量及其大小。它是一个数组，其中每个元素都是相应隐藏层的大小。
> 默认值为空数组，这意味着 AR-Net 将只有一层大小 n_forecasts
> 添加更多层会导致复杂性增加，从而增加计算时间。但是，增加的隐藏层数量可以帮助构建更复杂的关系。
> 为了在计算复杂性和提高精度之间进行权衡，建议将 ar_layers 设置为具有 1-2 个元素的数组。
> 然而，在大多数情况下，完全没有隐藏层可以实现足够好的性能。

lagged_reg_layers 定义了整个模型中滞后回归变量的 FFNN 的隐藏层数量及其大小,它是一个数组，其中每个元素都是相应隐藏层的大小.
> 默认值为空数组，这意味着滞后回归变量的 FFNN 将只有一层大小 n_forecasts 的最后一层。
> 添加更多层会导致复杂性增加，从而增加计算时间。但是，增加的隐藏层数量可以帮助构建更复杂的关系,这对于滞后回归变量特别有用。
> 为了在计算复杂性和提高精度之间进行权衡，建议将 lagged_reg_layers 设置为具有 1-2 个元素的数组。
> 然而，在大多数情况下，完全没有隐藏层可以实现足够好的性能。

注意，以前的 num_hidden_layers 和 d_hidden 参数现已弃用。现在，ar_net 和 covar_net 架构配置通过 ar_layers 和 lagged_reg_layers 输入。如果手动调整，建议的做法是为隐藏层的大小设置介于 n_lags 和 n_forecasts 之间的值。同样重要的是要注意，在当前的实现中，NeuralProphet 允许您为 ar_net 和 covar_net 中的隐藏层指定不同的大小。

# 4. 数据预处理相关参数

In [ ]:
normalize_y  在建模之前缩放时间序列。
> 默认情况下，NeuralProphet 对时间序列执行（软）最小-最大归一化。
> 如果序列值波动很大，归一化可以帮助模型训练过程。
> 但是，如果序列没有进行此类缩放，则用户可以关闭此功能或选择其他规范化

impute_missing 是关于插补给定序列中的缺失值
> 与 Prophet 类似，NeuralProphet 在没有 AR-Net 的情况下处于回归模式时也可以处理缺失值。
> 然而，当需要捕获自相关时，有必要对缺失值进行插补，因为这样建模就变成了一个有序问题。
> 在大多数情况下，让此参数处于默认值可以完美完成工作。

collect_metrics 用于选择要计算的指标的名称
> 它设置为 True，默认计算 mae 和 rms

# 5. 趋势相关参数

* 趋势灵活性（如果主要由 n_changepoints 控制，则设置趋势率可能变化的点数。此外，可以通过将 trend_reg 设置为大于零的值来规范趋势率变化。
* changepoints_range 控制用于拟合趋势的训练数据范围。 默认值 0.8 表示在最后 20% 的训练数据中未设置任何更改点。
* n_changepoints 是沿序列为趋势选择的变化点数。 默认为10
* changepoints 用于手动指定趋势更改点的日期。如果指定，则会忽略 n_changepoints 和 changepoints_range

# 6. 季节性相关参数

yearly_seasonality、weekly_seasonality 和 daily_seasonality 是关于要建模的季节性成分
> 例如，如果您使用温度数据，则可能可以选择“每日”和“每年”。例如，使用地铁的乘客数量更有可能具有每周的季节性。
> 将这些季节性设置为默认 auto 模式，让 NeuralProphet 根据可用数据量决定要包含哪些季节性。
> 例如，如果可用数据少于两年，则不会考虑年度季节性。同样，如果可用时间少于两周等，则不会考虑每周季节性......
> 但是，如果用户确定序列不包括年度、每周或每日季节性，因此模型不应被此类组件扭曲，则可以通过将相应的组件设置为 False 来显式关闭它们。
> 除此之外，参数 yearly_seasonality、weekly_seasonality 和 daily_seasonality 也可以设置为相应季节的傅里叶项数。每年默认值为 6 表示，每周默认值为 4，每日默认值为 6 。如果年度项数为 6，则实际上使年度季节性的傅里叶项总数为 12 （6*2），以同时容纳正弦项和余弦项。增加傅里叶项的数量可以使模型能够捕获相当复杂的季节性模式。然而，与 ar_layers 类似，这也会导致模型复杂性增加。用户可以通过查看最终的分量图来获得有关傅里叶项最佳数量的一些见解
> 默认 seasonality_mode 为“累加”。这意味着就季节性而言，该系列预计不会存在异方差。但是，如果序列包含明显的方差，其中季节性波动与趋势成正比，则 seasonality_mode 可以设置为乘法。


# 7. 正则化相关参数

 NeuralProphet还包含一系列正则化参数，用于控制模型系数并向模型引入稀疏性。这也有助于避免模型对训练数据的过拟合。

seasonality_reg参数，
> 默认值为0 
> 该参数用于控制季节性系数的正则化强度。
> 0.1-1范围内的小值允许拟合较大的季节性波动。
> 而1-100范围内的大值会对傅里叶系数施加更重的惩罚，从而抑制季节性波动。


ar_sparsity与n_lags参数可结合用于数据探索和特征选择：借助AR-Net的可扩展性，您可以设置较多的滞后项数量，并通过稀疏性识别过去时间步对预测精度的重要影响

对于future_regressor_regularization、event_regularization和country_holiday_regularization参数，其取值区间同样为0-1，概念与ar_sparsity一致。您可以根据不同回归量和事件需要抑制的程度，为它们分别设置不同的正则化参数。